# Process, Generate and Save Data

In [1]:
from utils import *

In [2]:
%%time
df = csvs_to_df()

CPU times: user 1min 50s, sys: 22 s, total: 2min 12s
Wall time: 4min 21s


In [3]:
%%time
df = process_df(df)

CPU times: user 14.6 s, sys: 4.99 s, total: 19.6 s
Wall time: 42.1 s


In [4]:
df

,timestamp,event,distinct_id,url,browser,browser_version,device,initial_referrer,initial_referring_domain,os,...,categories,category_scores,question_labels,question_scores,total_score,total_score_normalized,mp_country_code,city,radio,region
0,2020-04-07 04:23:37,Page Load,e0ec2fbc-e18a-4072-9a83-dc4b74a96579,/logout/,Chrome,80.0,NaN,$direct,$direct,Mac OS X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-07 10:19:52,Assessments Received,54b836dd-53da-48d6-aca7-9bad43ba6713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-07 10:19:52,Signup,54b836dd-53da-48d6-aca7-9bad43ba6713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-07 10:19:53,Page Load,54b836dd-53da-48d6-aca7-9bad43ba6713,/accounts/profile/patient/complete/,Chrome,80.0,NaN,$direct,$direct,Mac OS X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-07 10:47:06,Complete Profile,54b836dd-53da-48d6-aca7-9bad43ba6713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14934709,2022-03-31 23:59:40,App Page Load,74ACAFD1-99B7-4F2A-ACFD-0981D5632248,NaN,NaN,NaN,NaN,NaN,NaN,iOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LTE,NaN
14934710,2022-03-31 23:59:40,Screen Load,74ACAFD1-99B7-4F2A-ACFD-0981D5632248,NaN,NaN,NaN,NaN,NaN,NaN,iOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LTE,NaN
14934711,2022-03-31 23:59:40,Self Assessment Skipped,74ACAFD1-99B7-4F2A-ACFD-0981D5632248,NaN,NaN,NaN,NaN,NaN,NaN,iOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LTE,NaN
14934712,2022-03-31 23:59:40,App Page Load,74ACAFD1-99B7-4F2A-ACFD-0981D5632248,NaN,NaN,NaN,NaN,NaN,NaN,iOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LTE,NaN


In [5]:
df.columns

Index(['timestamp', 'event', 'distinct_id', 'url', 'browser',
       'browser_version', 'device', 'initial_referrer',
       'initial_referring_domain', 'os', 'referrer', 'referring_domain',
       'screen_height', 'screen_width', 'url_name', 'response_id', 'slug',
       'source', 'language', 'organization', 'organization_name', 'categories',
       'category_scores', 'question_labels', 'question_scores', 'total_score',
       'total_score_normalized', 'mp_country_code', 'city', 'radio', 'region'],
      dtype='object')

In [6]:
%%time
df_assess = get_df_assess(df)

CPU times: user 13.2 s, sys: 1.3 s, total: 14.5 s
Wall time: 50.1 s


In [7]:
df_assess.columns

Index(['timestamp', 'distinct_id', 'response_id', 'url_name', 'categories',
       'category_scores', 'question_labels', 'question_scores', 'total_score',
       'phq_suicide', 'phq_gad_1', 'phq_gad_2', 'phq_dep_1', 'phq_dep_2',
       'wemwbs_1', 'wemwbs_2', 'wemwbs_3', 'wemwbs_4', 'wemwbs_5', 'wemwbs_6',
       'wemwbs_7', 'wemwbs_8', 'wemwbs_9', 'wemwbs_10', 'wemwbs_11',
       'wemwbs_12', 'wemwbs_13', 'wemwbs_14', 'retired', 'wsas_1', 'wsas_2',
       'wsas_3', 'wsas_4', 'wsas_5', 'gs_mood_score', 'gs_wemwbs_score',
       'gs_wsas_score', 'phq_gad_score', 'phq_dep_score', 'phq_mood_score',
       'wemwbs_score', 'wsas_score'],
      dtype='object')

In [8]:
file_name = 'mixpanel_assessment_complete_events.csv'
save_pd_to_csv(df_assess, file_name)

In [9]:
%%time
df_assess_fl = get_first_last_assess(df_assess)

CPU times: user 4.51 s, sys: 3.65 s, total: 8.16 s
Wall time: 9.07 s


In [10]:
df_assess_fl

,distinct_id,timestamp_first,response_id_first,url_name_first,categories_first,category_scores_first,question_labels_first,question_scores_first,total_score_first,phq_suicide_first,...,dep_diff,anx_diff,mood_diff,func_diff,wb_diff,dep_d,anx_d,mood_d,func_d,wb_d
0,000082ab-c331-48f7-a4e3-7cd1dd7daae5,2020-08-12 05:42:12,1ab911b8-f108-4a6d-93de-0f9ab5f3f36c,NaN,"[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",72.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0000b5a9-a906-4e38-aa73-8416f6b0385a,2020-12-28 23:22:36,65101fc3-bb6b-480f-aa00-c3f1e2d2f535,NaN,"[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",94.0,3.0,...,-2.0,0.0,-2.0,0.0,1.0,-1.843472,0.000000,-1.052851,0.000000,0.175960
2,0000cbfe-c429-4547-9c3e-06e9ec366bbd,2022-01-19 11:51:05,abfd64c6-ffc0-4c6a-9ce4-4adaf4863c99,NaN,"[Mood, Well-Being, Functioning]","[2, 23, 22]","[3, 0, 2, 0, 0, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, ...","[3, 0, 2, 0, 0, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, ...",50.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,0000e58e-3f7b-4b23-af42-2af3253a3950,2021-05-09 09:52:11,06383e4d-b4de-4b77-840f-c4ddf7c34d2c,NaN,"[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000fb50-a065-4082-9baf-374e85ee5e42,2022-01-14 16:33:25,0651fc79-82b8-44b0-b2ab-8c5aeef734e4,NaN,"[Mood, Well-Being, Functioning]","[8, 36, 28]","[2, 2, 2, 2, 2, 3, 3, 2, 3, 4, 4, 2, 2, 2, 1, ...","[2, 2, 2, 2, 2, 3, 3, 2, 3, 4, 4, 2, 2, 2, 1, ...",74.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159938,fffd12df-bc74-4d4d-964d-affef487ab75,2020-07-30 02:23:18,c4ee57bc-3a18-448b-b9f7-4aa530b62ac9,NaN,"[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159939,fffd89db-6dec-454b-afc0-624d420c3317,2020-06-10 11:37:17,e349c599-c0ca-47d9-8def-d88a9f917d72,NaN,"[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159940,fffec8b3-1f66-494a-a7ce-421c982dee38,2022-01-14 22:44:30,eeec9669-c4b4-44b2-96c7-e50c398cb61f,NaN,"[Mood, Well-Being, Functioning]","[5, 39, 13]","[3, 1, 2, 0, 2, 3, 3, 2, 1, 1, 3, 4, 3, 3, 3, ...","[3, 1, 2, 0, 2, 3, 3, 2, 1, 1, 3, 4, 3, 3, 3, ...",60.0,3.0,...,-3.0,-1.0,-4.0,-1.0,6.0,-2.765208,-0.918479,-2.105702,-0.201723,1.055763
159941,ffff0ec5-b479-4d60-b16e-23267134b198,2021-01-24 22:35:04,ae507450-ab55-47ca-992f-9d0be7dec065,NaN,"[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_assess_fl.columns

Index(['distinct_id', 'timestamp_first', 'response_id_first', 'url_name_first',
       'categories_first', 'category_scores_first', 'question_labels_first',
       'question_scores_first', 'total_score_first', 'phq_suicide_first',
       'phq_gad_1_first', 'phq_gad_2_first', 'phq_dep_1_first',
       'phq_dep_2_first', 'wemwbs_1_first', 'wemwbs_2_first', 'wemwbs_3_first',
       'wemwbs_4_first', 'wemwbs_5_first', 'wemwbs_6_first', 'wemwbs_7_first',
       'wemwbs_8_first', 'wemwbs_9_first', 'wemwbs_10_first',
       'wemwbs_11_first', 'wemwbs_12_first', 'wemwbs_13_first',
       'wemwbs_14_first', 'retired_first', 'wsas_1_first', 'wsas_2_first',
       'wsas_3_first', 'wsas_4_first', 'wsas_5_first', 'gs_mood_score_first',
       'gs_wemwbs_score_first', 'gs_wsas_score_first', 'phq_gad_score_first',
       'phq_dep_score_first', 'phq_mood_score_first', 'wemwbs_score_first',
       'wsas_score_first', 'timestamp_last', 'response_id_last',
       'url_name_last', 'categories_last', 'cate

In [12]:
file_name = 'mixpanel_users_assess_scores.csv'
save_pd_to_csv(df_assess_fl, file_name)

In [13]:
%%time
df_use_resource = get_df_use_resource(df)

CPU times: user 2.24 s, sys: 119 ms, total: 2.35 s
Wall time: 7.98 s


In [14]:
file_name = 'mixpanel_use_resource_events.csv'
save_pd_to_csv(df_use_resource, file_name)

In [15]:
%%time
df_accesses = get_df_accesses(df_use_resource)

CPU times: user 38.5 s, sys: 646 ms, total: 39.1 s
Wall time: 43.9 s


In [16]:
file_name = 'mixpanel_use_resource_unduplicated_events.csv'
save_pd_to_csv(df_accesses, file_name)

In [17]:
%%time
df_first_access = get_df_first_access(df_accesses)

CPU times: user 3.09 s, sys: 193 ms, total: 3.28 s
Wall time: 8.2 s


In [18]:
file_name = 'df_first_access.csv'
save_pd_to_csv(df_first_access, file_name)

In [19]:
%%time
df_first_access_merged = get_df_first_access_merged(df, df_accesses, df_first_access)

CPU times: user 41 s, sys: 3.37 s, total: 44.4 s
Wall time: 45.3 s


In [20]:
file_name = 'mixpanel_users_outerjoin.csv'
save_pd_to_csv(df_first_access_merged, file_name)

In [21]:
%%time
df_access_count = get_df_access_count(df_accesses)

CPU times: user 1.49 s, sys: 79.8 ms, total: 1.57 s
Wall time: 1.57 s


In [22]:
file_name = 'df_access_count.csv'
save_pd_to_csv(df_access_count, file_name)

In [23]:
%%time
df_assess_fl_access = get_df_assess_fl_access(df_assess_fl, df_access_count)

CPU times: user 1.07 s, sys: 39.9 ms, total: 1.11 s
Wall time: 1.4 s


In [24]:
file_name = 'mixpanel_users_assessments.csv'
save_pd_to_csv(df_assess_fl_access, file_name)